# Wiley Handlabeled Data Splitter
This notebook focusses on building functions that enable the taking-in of a list or a directory of fulltext .txt files, and splitting them into individual tokens to be hand-labeled by humans, so they can be used as training data for NLP models.

In [1]:
demo_file = "10.1002_ange.201500925.txt"

In [2]:
filepath = '/Users/Jonathan/Desktop/Test_Folder/'

In [2]:
import nltk
from nltk import tokenize

In [3]:
import os
import random
import pandas as pd
from openpyxl import load_workbook

In [4]:
def append_df_to_excel(filename, df, sheet_name='Sheet1', startrow=0, startcol=None,
                       truncate_sheet=False,
                       **to_excel_kwargs):
    """
    Append a DataFrame [df] to existing Excel file [filename]
    into [sheet_name] Sheet.
    If [filename] doesn't exist, then this function will create it.

    Parameters:
      filename : File path or existing ExcelWriter
                 (Example: '/path/to/file.xlsx')
      df : dataframe to save to workbook
      sheet_name : Name of sheet which will contain DataFrame.
                   (default: 'Sheet1')
      startrow : upper left cell row to dump data frame.
                 Per default (startrow=None) calculate the last row
                 in the existing DF and write to the next row...

      startcol : upper left cell column to dump data frame.


      truncate_sheet : truncate (remove and recreate) [sheet_name]
                       before writing DataFrame to Excel file
      to_excel_kwargs : arguments which will be passed to `DataFrame.to_excel()`
                        [can be dictionary]

    Returns: None
    """

    print('On dataframe ', int(startcol/6)+1)
    # ignore [engine] parameter if it was passed
    if 'engine' in to_excel_kwargs:
        to_excel_kwargs.pop('engine')

    writer = pd.ExcelWriter(filename, engine='openpyxl')

    # Python 2.x: define [FileNotFoundError] exception if it doesn't exist
    try:
        FileNotFoundError
    except NameError:
        FileNotFoundError = IOError


    try:
        # try to open an existing workbook
        writer.book = load_workbook(filename)

        if startcol is None and sheet_name in writer.book.sheetnames:
            startcol = writer.book[sheet_name].max_col

        # truncate sheet
        if truncate_sheet and sheet_name in writer.book.sheetnames:
            # index of [sheet_name] sheet
            idx = writer.book.sheetnames.index(sheet_name)
            # remove [sheet_name]
            writer.book.remove(writer.book.worksheets[idx])
            # create an empty sheet [sheet_name] using old index
            writer.book.create_sheet(sheet_name, idx)

        # copy existing sheets
        writer.sheets = {ws.title:ws for ws in writer.book.worksheets}
    except FileNotFoundError:
        # file does not exist yet, we will create it
        pass


    if startcol is None:
        startcol = 0

    # write out the new sheet
    df.to_excel(writer, sheet_name, startrow=startrow, startcol=startcol, **to_excel_kwargs)

    # save the workbook
    writer.save()

In [5]:
directory_path = '/Users/Jonathan/Desktop/WileySynthCorpus/'
name_list = ['name', 'tokens', 'BESIO', 'entity', 'mol_class']

## Note: 
I highly recommend that you use a smaller number of papers per sheet, or write a loop that makes several different spreadsheets. It seems like this gets slower and slower the more papers we add into the spreadsheet.

In [14]:
#Ok, that would make for a perfect text. Now we need to put that perfect text into a spreadsheet. 
#We'll then plan to generalize to multiple text files in a single directory.
# directory_path = '/Users/Jonathan/Desktop/Test_Folder/'
# name_list = ['name', 'tokens', 'BESIO', 'entity', 'mol_class']
publication_lists = os.listdir(directory_path)

#Need to long-term update this to randomly select a subset, based on how many different papers they want to 
#add, and how many papers per sheet is selected.
papers_per_sheet = 10
total_papers = 40
num_sheets = total_papers/papers_per_sheet
selected_pubs = []
filename = "Wiley_Labeling_Spreadsheet.xlsx"

#This loop randomly selects papers from our total corpus list
while total_papers > len(selected_pubs):
    random_num = random.randint(1, len(publication_lists))
    print(random_num)
    selected_pubs.append(publication_lists[random_num-1])
#print(selected_pubs)
    

#Ok, the above definitely is working totally fine. 
pubs_in_sheet = 0
total_pubs = 0
for i in selected_pubs:
    #Make it such that the sheet number we're on is always an int.
    sheet_number = int(total_pubs/papers_per_sheet)
    #Check to reset column count:
    if pubs_in_sheet%papers_per_sheet == 0:
        pubs_in_sheet = 0
    #Open the file, clean it of odd characters, then tokenize it with NLTK.
    #This will definitely fail if it tries to open the json files in this directory
    #But otherwise, it's fine. 2/25000, what're the odds?
    file = open(directory_path+str(i), mode = 'r')
    text = file.read()
    file.close()
    text = text.replace("\xad", '-')
    cleaned = text.replace("\n", " ")
    cleaned = cleaned.replace('\x0c', ' ')
    #Double space to single space
    cleaned = cleaned.replace('  ', ' ')
    #Triple space to single space
    cleaned = cleaned.replace('   ', ' ')
    #Tokenize the raw text at a word level.
    tokenized_texts = tokenize.word_tokenize(cleaned)
    
    #Build a new dataframe and place the tokens and doi into it    
    df = pd.DataFrame(columns = name_list)
    df['tokens'] = tokenized_texts
    #Add the doi, which is just the filename minus the .txt extension
    df['name'][3] = str(i[:-4])
    df['name'][4] = "Paper Count: " + str(pubs_in_sheet)
    
    append_df_to_excel(directory_path+filename, df, sheet_name = "Sheet "+str(sheet_number), startcol = 6*pubs_in_sheet)
    pubs_in_sheet += 1
    total_pubs += 1
print("Your spreadsheet is ready!")

19950
1796
938
22363
21240
25111
24121
24854
18473
23766
21519
19901
14452
11766
6123
8938
10910
25103
5794
617
9627
21367
6437
23322
18118
13038
20399
20356
14337
4037
17911
15886
16551
17878
9668
21714
24524
1423
19250
23910
On dataframe  1
On dataframe  2
On dataframe  3
On dataframe  4
On dataframe  5
On dataframe  6
On dataframe  7
On dataframe  8
On dataframe  9
On dataframe  10
On dataframe  1
On dataframe  2
On dataframe  3
On dataframe  4
On dataframe  5
On dataframe  6
On dataframe  7
On dataframe  8
On dataframe  9
On dataframe  10
On dataframe  1
On dataframe  2
On dataframe  3
On dataframe  4
On dataframe  5
On dataframe  6
On dataframe  7
On dataframe  8
On dataframe  9
On dataframe  10
On dataframe  1
On dataframe  2
On dataframe  3
On dataframe  4
On dataframe  5
On dataframe  6
On dataframe  7
On dataframe  8


C:\Users\Jonathan\Anaconda3\lib\site-packages\ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


On dataframe  9
On dataframe  10
Your spreadsheet is ready!


In [12]:
#Check the pub.
publication_lists[19275]

'10.1002_jhet.5570310208.txt'

Now we functionalize it all:

In [22]:
def build_spreadsheets(filename, doc_directory_path, column_name_list, papers_per_sheet = 10, total_papers = 50):
    """"""
    
    publication_lists = os.listdir(directory_path)

    #Need to long-term update this to randomly select a subset, based on how many different papers they want to 
    #add, and how many papers per sheet is selected.
    num_sheets = total_papers/papers_per_sheet
    selected_pubs = []

    #This loop randomly selects papers from our total corpus list
    while total_papers > len(selected_pubs):
        random_num = random.randint(1, len(publication_lists))
        print("Random Publication Selected: ", (random_num-1))
        selected_pubs.append(publication_lists[random_num-1])


    #Ok, the above definitely is working totally fine. 
    pubs_in_sheet = 0
    total_pubs = 0
    for i in selected_pubs:
        #Make it such that the sheet number we're on is always an int.
        sheet_number = int(total_pubs/papers_per_sheet)
        #Check to reset column count:
        if pubs_in_sheet%papers_per_sheet == 0:
            pubs_in_sheet = 0
        #Open the file, clean it of odd characters, then tokenize it with NLTK.
        #This will definitely fail if it tries to open the json files in this directory
        #But otherwise, it's fine. 2/25000, what're the odds?
        file = open(directory_path+str(i), mode = 'r')
        text = file.read()
        file.close()
        text = text.replace("\xad", '-')
        cleaned = text.replace("\n", " ")
        cleaned = cleaned.replace('\x0c', ' ')
        #Double space to single space
        cleaned = cleaned.replace('  ', ' ')
        #Triple space to single space
        cleaned = cleaned.replace('   ', ' ')
        #Tokenize the raw text at a word level.
        tokenized_texts = tokenize.word_tokenize(cleaned)

        #Build a new dataframe and place the tokens and doi into it    
        df = pd.DataFrame(columns = name_list)
        df['tokens'] = tokenized_texts
        #Add the doi, which is just the filename minus the .txt extension
        df['name'][3] = str(i[:-4])
        df['name'][4] = "Paper Count: " + str(pubs_in_sheet)

        append_df_to_excel(directory_path+filename, df, sheet_name = "Sheet "+str(sheet_number), startcol = 6*pubs_in_sheet)
        pubs_in_sheet += 1
        total_pubs += 1
    print("Your spreadsheet is ready!")

In [23]:
i = 0
col_list = ['name', 'tokens', 'BESIO', 'entity', 'mol_class']
while i < 10:
    print("Wiley_Spreadsheet {}.xlsx".format(i))
    build_spreadsheets("Wiley_Spreadsheet {}.xlsx".format(str(i)), '/Users/Jonathan/Desktop/WileySynthCorpus/', col_list)
    i += 1

Wiley_Spreadsheet 0.xlsx
Random Publication Selected:  7750
Random Publication Selected:  18018
Random Publication Selected:  24960
Random Publication Selected:  20523
Random Publication Selected:  7247
Random Publication Selected:  392
Random Publication Selected:  22032
Random Publication Selected:  12017
Random Publication Selected:  3112
Random Publication Selected:  22024
Random Publication Selected:  2819
Random Publication Selected:  4968
Random Publication Selected:  18232
Random Publication Selected:  5188
Random Publication Selected:  13589
Random Publication Selected:  2585
Random Publication Selected:  18563
Random Publication Selected:  3483
Random Publication Selected:  22227
Random Publication Selected:  403
Random Publication Selected:  21197
Random Publication Selected:  8014
Random Publication Selected:  6194
Random Publication Selected:  8333
Random Publication Selected:  3047
Random Publication Selected:  19015
Random Publication Selected:  10864
Random Publication 

C:\Users\Jonathan\Anaconda3\lib\site-packages\ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


On dataframe  6
On dataframe  7
On dataframe  8
On dataframe  9
On dataframe  10
On dataframe  1
On dataframe  2
On dataframe  3
On dataframe  4
On dataframe  5
On dataframe  6
On dataframe  7
On dataframe  8
On dataframe  9
On dataframe  10
On dataframe  1
On dataframe  2
On dataframe  3
On dataframe  4
On dataframe  5
On dataframe  6
On dataframe  7
On dataframe  8
On dataframe  9
On dataframe  10
Your spreadsheet is ready!
Wiley_Spreadsheet 1.xlsx
Random Publication Selected:  7323
Random Publication Selected:  15433
Random Publication Selected:  20127
Random Publication Selected:  9056
Random Publication Selected:  16153
Random Publication Selected:  4019
Random Publication Selected:  18182
Random Publication Selected:  16414
Random Publication Selected:  8577
Random Publication Selected:  24953
Random Publication Selected:  13121
Random Publication Selected:  17036
Random Publication Selected:  12404
Random Publication Selected:  16297
Random Publication Selected:  25173
Random Pu